## Menghitung Jarak(buka,tutup) audio rekaman langsung dengan data yang sudah disimpan

### Install library 

In [2]:
!pip install dtaidistance librosa numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.9 MB/s  0:00:00


### import library

In [3]:
import os
import numpy as np
import librosa
from dtaidistance import dtw


### Fungsi Ekstraksi MFCC

In [4]:
def extract_mfcc(file_path, n_mfcc=20):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T   # time × feature


### Load dataset audio buka/tutup

In [6]:
dataset_path = "Gabungan_Suara"

data_mfcc = []
labels = []

for label in ["buka", "tutup"]:
    folder = os.path.join(dataset_path, label)
    for file in os.listdir(folder):
        if file.endswith(".wav") or file.endswith(".mp3"):
            path = os.path.join(folder, file)
            mfcc = extract_mfcc(path)
            data_mfcc.append(mfcc)
            labels.append(label)

print("Jumlah data:", len(data_mfcc))


Jumlah data: 400


### Load rekaman input antara buka dan tutup

In [18]:
rekaman_path = "Rekaman_baru .wav"   # ganti nama file kamu
print("Rekaman ini adalah: BUKA (sesuai input pengguna)\n")

mfcc_input = extract_mfcc(rekaman_path)

# Hitung jarak dengan tiap data
distances = [dtw_distance(mfcc_input, mfcc_data) for mfcc_data in data_mfcc]

min_idx = np.argmin(distances)
hasil_prediksi = labels[min_idx]

print("=== HASIL DTW ===")
for i, d in enumerate(distances):
    print(f"{labels[i]} → jarak = {d:.2f}")

print("\n>>> HASIL PREDIKSI DTW:", hasil_prediksi.upper())

# Koreksi apakah cocok atau tidak
if hasil_prediksi == "buka":
    print("✓ Cocok! Rekaman benar adalah BUKA.")
else:
    print("✗ Tidak cocok! DTW memprediksi TUTUP padahal ini BUKA.")


Rekaman ini adalah: BUKA (sesuai input pengguna)

=== HASIL DTW ===
buka → jarak = 1280.95
buka → jarak = 1788.55
buka → jarak = 1112.20
buka → jarak = 1466.73
buka → jarak = 1633.98
buka → jarak = 1714.61
buka → jarak = 1913.92
buka → jarak = 857.23
buka → jarak = 875.05
buka → jarak = 1218.96
buka → jarak = 1137.55
buka → jarak = 1098.30
buka → jarak = 1527.10
buka → jarak = 1120.85
buka → jarak = 1077.61
buka → jarak = 1290.51
buka → jarak = 1310.17
buka → jarak = 1093.38
buka → jarak = 1665.44
buka → jarak = 951.97
buka → jarak = 1353.62
buka → jarak = 1711.96
buka → jarak = 1115.91
buka → jarak = 1262.22
buka → jarak = 1866.18
buka → jarak = 1068.64
buka → jarak = 1315.56
buka → jarak = 889.89
buka → jarak = 1359.03
buka → jarak = 1105.39
buka → jarak = 1556.92
buka → jarak = 2320.83
buka → jarak = 1195.50
buka → jarak = 1542.52
buka → jarak = 1245.89
buka → jarak = 1529.23
buka → jarak = 1081.88
buka → jarak = 1003.53
buka → jarak = 1705.33
buka → jarak = 1014.03
buka → jarak = 1

In [19]:
rekaman_path = "rekaman_tutup.wav"   # ganti nama file kamu
print("Rekaman ini adalah: TUTUP (sesuai input pengguna)\n")

mfcc_input = extract_mfcc(rekaman_path)

# Hitung jarak dengan tiap data
distances = [dtw_distance(mfcc_input, mfcc_data) for mfcc_data in data_mfcc]

min_idx = np.argmin(distances)
hasil_prediksi = labels[min_idx]

print("=== HASIL DTW ===")
for i, d in enumerate(distances):
    print(f"{labels[i]} → jarak = {d:.2f}")

print("\n>>> HASIL PREDIKSI DTW:", hasil_prediksi.upper())

# Koreksi apakah cocok atau tidak
if hasil_prediksi == "tutup":
    print("✓ Cocok! Rekaman benar adalah TUTUP.")
else:
    print("✗ Tidak cocok! DTW memprediksi BUKA padahal ini TUTUP.")


Rekaman ini adalah: TUTUP (sesuai input pengguna)

=== HASIL DTW ===
buka → jarak = 1202.29
buka → jarak = 1628.56
buka → jarak = 901.32
buka → jarak = 1296.36
buka → jarak = 1457.85
buka → jarak = 1517.66
buka → jarak = 1657.59
buka → jarak = 817.87
buka → jarak = 824.68
buka → jarak = 1012.40
buka → jarak = 855.76
buka → jarak = 983.98
buka → jarak = 1280.76
buka → jarak = 975.62
buka → jarak = 983.13
buka → jarak = 1204.59
buka → jarak = 1142.18
buka → jarak = 1042.18
buka → jarak = 1587.99
buka → jarak = 867.63
buka → jarak = 1139.86
buka → jarak = 1519.28
buka → jarak = 866.15
buka → jarak = 1058.78
buka → jarak = 1661.71
buka → jarak = 881.44
buka → jarak = 1132.64
buka → jarak = 851.64
buka → jarak = 1163.65
buka → jarak = 1008.29
buka → jarak = 1358.51
buka → jarak = 2097.54
buka → jarak = 981.17
buka → jarak = 1362.01
buka → jarak = 978.97
buka → jarak = 1359.47
buka → jarak = 995.37
buka → jarak = 867.71
buka → jarak = 1557.99
buka → jarak = 800.39
buka → jarak = 849.42
buka 

### Hitung DTW (meggunakan DTAIDistance)

In [16]:
from dtaidistance import dtw_ndim

def dtw_distance(m1, m2):
    return dtw_ndim.distance(m1, m2)

distances = [dtw_distance(mfcc_input, mfcc_data) for mfcc_data in data_mfcc]


In [17]:
min_idx = np.argmin(distances)
print("Hasil perbandingan DTW:")

for i, d in enumerate(distances):
    print(f"{labels[i]} → jarak = {d:.2f}")

print("\n>>> Hasil Terdekat:", labels[min_idx].upper())


Hasil perbandingan DTW:
buka → jarak = 1280.95
buka → jarak = 1788.55
buka → jarak = 1112.20
buka → jarak = 1466.73
buka → jarak = 1633.98
buka → jarak = 1714.61
buka → jarak = 1913.92
buka → jarak = 857.23
buka → jarak = 875.05
buka → jarak = 1218.96
buka → jarak = 1137.55
buka → jarak = 1098.30
buka → jarak = 1527.10
buka → jarak = 1120.85
buka → jarak = 1077.61
buka → jarak = 1290.51
buka → jarak = 1310.17
buka → jarak = 1093.38
buka → jarak = 1665.44
buka → jarak = 951.97
buka → jarak = 1353.62
buka → jarak = 1711.96
buka → jarak = 1115.91
buka → jarak = 1262.22
buka → jarak = 1866.18
buka → jarak = 1068.64
buka → jarak = 1315.56
buka → jarak = 889.89
buka → jarak = 1359.03
buka → jarak = 1105.39
buka → jarak = 1556.92
buka → jarak = 2320.83
buka → jarak = 1195.50
buka → jarak = 1542.52
buka → jarak = 1245.89
buka → jarak = 1529.23
buka → jarak = 1081.88
buka → jarak = 1003.53
buka → jarak = 1705.33
buka → jarak = 1014.03
buka → jarak = 1141.43
buka → jarak = 1076.82
buka → jarak =